# BSD68 Dataset Denoising tramite Noise2Void

Questo notebook descrive, passo dopo passo, l'addestramento e il testing di una **Rete Neurale U-Net** per la riduzione del rumore sulle immagini del **Dataset BSD68**.  

La rete applica l'algoritmo **Noise2Void (N2V)**, la cui implementazione è resa disponibile dalla libreria **CAREamics**, permettendo di ottenere immagini pulite **senza necessità di Ground Truth**.

L'obiettivo di questa attività è:

- Comprendere le potenzialità delle **Deep Neural Networks** nell'ambito del denoising.  
- Confrontare i risultati di N2V con quelli ottenuti tramite metodi di denoising tradizionali.  
- Approfondire il funzionamento e le caratteristiche dell'algoritmo **Noise2Void**.

## Il Dataset BSD68

Il **BSD68** è un dataset di riferimento per la ricerca nel denoising. Le sue caratteristiche principali sono:

- Contiene immagini **in bianco e nero** di scene naturali, animali, persone e oggetti.  
- Le immagini sono state **artificialmente corrotte da rumore** di diversa intensità.  
- Sono disponibili le **Ground Truth**, ossia le immagini originali pulite, utili per valutare la qualità del denoising.  

Grazie alla presenza di Ground Truth, possiamo **valutare in modo oggettivo** le capacità dell'algoritmo **Noise2Void**, osservando come riesca a recuperare dettagli significativi pur lavorando su immagini rumorose.


Il BSD68 permette di testare Noise2Void su soggetti facilmente riconoscibili, offrendo una **valutazione visiva immediata** e quantitativa delle prestazioni dell'algoritmo.  
Questo lo rende ideale sia per esperimenti didattici sia per benchmarking nel contesto della ricerca scientifica.


# Download del Dataset

Il dataset viene scaricato utilizzando il **Portfolio** reso disponibile dalla libreria **CAREamics**.  
Contiene più di **3.000 immagini** di formati e soggetti differenti, salvate in formato **.TIFF** con una codifica speciale compatibile con **ImageJ**.


## Struttura del Dataset

Le immagini sono già suddivise in **4 categorie principali**:

1. **Dati di Training** – 3168 immagini  
   Consentono alla rete neurale di **apprendere i parametri ottimali** durante l'addestramento.

2. **Dati di Validation** – 4 immagini  
   Permettono di **monitorare l’andamento dell’addestramento**, evitando il rischio di overfitting.

3. **Dati di Testing** – 68 immagini  
   Utilizzati **solo a fine addestramento** per valutare le prestazioni finali del modello.

4. **Ground Truth dei Dati di Testing** – 68 immagini  
   Contengono le versioni **pulite** dei dati di testing, necessarie per il **confronto quantitativo e visivo** dei risultati.

In [ ]:
import tifffile
import sys
import os
import numpy as np
import matplotlib.pyplot as plt
from pathlib import Path
from careamics import CAREamist
from careamics.config import create_n2v_configuration
from careamics.utils.metrics import scale_invariant_psnr

sys.path.append('library')

import library.dataset as dataset

In [ ]:
# Downlaod the Dataset
root_path = Path("notebooks/data/bsd68")
dataset.load_bsd68_dataset(root_path)

# The Dataset is already split into Training, Validation, Testing and Grand Truths
data_path = Path(root_path / "denoising-N2V_BSD68.unzip/BSD68_reproducibility_data")
train_path = data_path / "train"
val_path = data_path / "val"
test_path = data_path / "test" / "images"
gt_path = data_path / "test" / "gt"


## Visualizzazione del Dataset

Per comprendere meglio le caratteristiche del BSD68 Dataset, visualizziamo alcune immagini a scopo illustrativo.

Si può notare che alcune immagini presentano trasformazioni come **simmetrie, rotazioni e flip**, tipiche delle tecniche di **data augmentation** utilizzate durante l’addestramento delle reti U-Net.  
Queste trasformazioni aiutano la Rete Neurale a diventare più **robusta**, permettendole di gestire meglio situazioni diverse da quelle presenti nel dataset originale.

Di seguito mostriamo alcune immagini scelte casualmente dal Dataset, evidenziando le varie tipologie di soggetti e di rumore applicato.


In [ ]:
train_images = tifffile.imread(next(iter(train_path.rglob("*.tiff")))) # 3168 images
val_images = tifffile.imread(next(iter(val_path.rglob("*.tiff"))))
starting_index = 8 #Change this to show different images

fig, axes = plt.subplots(nrows=4, ncols=4, figsize=(15, 15))

for ax, idx in zip(axes.flat, range(starting_index, starting_index + 16)): #Couples the axes with the indexes
    ax.imshow(train_images[idx], cmap="gray")
    ax.set_title(f"Training Image {idx}")
    
plt.tight_layout()
plt.show()

## Creazione della Configurazione di Training

Prima di procedere all'addestramento della rete, è necessario creare un oggetto di **configurazione** che definisca sia l'architettura della rete sia i parametri di training.

I parametri principali da considerare sono:

- **batch_size**: il numero di immagini elaborate in parallelo durante ogni step di addestramento.  
  Valori più alti richiedono più memoria GPU, ma possono velocizzare l’addestramento.
- **num_epochs**: il numero di volte in cui l’intero dataset viene utilizzato per aggiornare i pesi della rete.


In [ ]:
config = create_n2v_configuration(
    experiment_name="bsd68_n2v",
    data_type="tiff",
    axes="SYX",
    patch_size=(64, 64),
    batch_size=64,
    num_epochs=10,
)

print(config)

## Addestramento del Modello
L'addestramento del modello avviene tramite la creazione di un oggetto **CAREamist**, utilizzando la configurazione definita precedentemente.

### Come funziona:

- Il metodo `.train()` richiede i **percorsi alle immagini di Training e Validation**.
- Durante l'addestramento, la rete apprende a **ridurre il rumore** dalle immagini, aggiornando i propri pesi.
- È possibile osservare in tempo reale alcuni parametri di performance, come:
  - **Loss**: misura l'errore tra le predizioni del modello e i valori target.
  - **PSNR (Peak Signal-to-Noise Ratio)**: indica la qualità delle immagini denoised rispetto alla ground truth (se disponibile).

In [ ]:
# Before proceding, make sure your GPU is available to PyTorch or the training will be very slow

careamist = CAREamist(source=config, work_dir="notebooks/models/bsd68")

# train model
print(f"Training starting now...")
careamist.train(train_source=train_path, val_source=val_path)
print("Training ended!")

## Generate Predictions

Terminato l'addestramento, possiamo osservare il comportamento del modello chiedendogli di **predire le immagini pulite** a partire dai campioni rumorosi del set **Testing**.

### Procedura:

- Utilizziamo il metodo `.predict` del modello CAREamist.
- Il modello impiega l'**ultimo checkpoint salvato** durante il training.
- Le predizioni vengono effettuate su tutte le **68 immagini** del dataset BSD68 Testing.

Questo passaggio permette di ottenere le immagini denoised, che saranno poi confrontate con le Ground Truth per valutare l’efficacia dell’algoritmo Noise2Void.


In [ ]:
output_path = "notebooks/predictions/bsd68/predictions.tiff"

prediction = careamist.predict(
    source=test_path,
    axes="YX",
    tile_size=(128, 128),
    tile_overlap=(48, 48),
    batch_size=1,
)

## Visualizzazione delle Predizioni

A questo punto possiamo mostrare i risultati delle predizioni del modello.

### Procedura:

- Selezioniamo **n immagini casuali** dal dataset di test.
- Visualizziamo:
  - **Input rumoroso**
  - **Predizione della rete (denoised)**
  - **Ground Truth**, quando disponibile, per un confronto diretto.
  
Questo confronto visivo consente di valutare in modo intuitivo:

- La capacità dell'algoritmo **Noise2Void (N2V)** di rimuovere il rumore.
- I limiti del modello in determinate situazioni o tipologie di rumore.

In [ ]:
n = 4

test_images = [tifffile.imread(f) for f in sorted(test_path.glob("*.tiff"))]
ground_truth_images = [tifffile.imread(f) for f in sorted(gt_path.glob("*.tiff"))]

random_indexes = np.random.choice(range(len(test_images)), n)

fig, ax = plt.subplots(4, 3, figsize=(15, 15))
for a, i in zip(range(n), random_indexes):
    ax[a, 0].imshow(test_images[i], cmap="gray")
    ax[a, 0].set_title(f"Noisy {i}")
    ax[a, 1].imshow(prediction[i].squeeze(), cmap="gray")
    ax[a, 1].set_title(f"Prediction {i}")
    ax[a, 2].imshow(ground_truth_images[i], cmap="gray")
    ax[a, 2].set_title(f"Grand Truth {i}")
